In [4]:
import pandas as pd
import numpy as np

In [5]:
# Load the saved file
df = pd.read_parquet('../data/processed_sticker_dataset.parquet')

# Convert lists back to Numpy arrays of float32
df["combined_embedding"] = df["combined_embedding"].apply(lambda x: np.array(x, dtype=np.float32))

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38807 entries, 0 to 38806
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   combined_embedding  38807 non-null  object
 1   image_path          38807 non-null  object
dtypes: object(2)
memory usage: 606.5+ KB
